In [3]:
import pymongo
import pickle
import keywords as kw
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

### Keywords for a single document

In [ ]:
#FIND KEYWORDS FOR DOCUMENTS IN INDEX
with open('../speech_indexes.pickle', 'rb') as handle:
    speech_indexes = pickle.load(handle)
kw.extract_document_keywords(speech_indexes, database)

In [4]:
# READ KEYWORDS FROM DB AND SAVE THEM AS A PICKLE FILE
doc_keywords = list(database.find({ },{"_id":1, "keywords":1}))
#with open('keywords_per_speech.pickle', 'wb') as handle:
#    pickle.dump(doc_keywords, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Keywords per party

In [5]:
parties = database.distinct("political_party")

In [ ]:
keywords_by_party = kw.keywords_per_party(parties, database)

In [6]:
#SAVE KEYWORDS BY PARTY IN PICKLE FORMAT

#with open('keywords_by_party.pickle', 'wb') as handle:
#    pickle.dump(keywords_by_party, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
print(keywords_by_party['συνασπισμος ριζοσπαστικης αριστερας'])
print(keywords_by_party['ελληνικη λυση - κυριακος βελοπουλος'])
print(keywords_by_party['νεα δημοκρατια'])
print(keywords_by_party['ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια'])
print(keywords_by_party['ενωση κεντρωων'])

{'δημοκρατ': 4994, 'νε': 4927, 'τροπολογ': 2902, 'συριζ': 2291, 'ευρ': 2235, 'συνεδριας': 1965, 'σχεδ': 1944, 'ανθρωπ': 1943, 'αναπτυξ': 1925, 'εργας': 1894, 'υγει': 1867, 'συστημ': 1847, 'αρχ': 1836, 'σωμ': 1826, 'δικαιωμ': 1823, 'ερωτης': 1768, 'εργαζομεν': 1736, 'μετρ': 1698}
{'δημοκρατ': 263, 'νε': 251, 'συριζ': 164, 'λυς': 140, 'ευρ': 137, 'κυκλ': 111, 'ανθρωπ': 111, 'συνεδριας': 89, 'ωρ': 85, 'ερωτης': 83, 'σωμ': 81, 'παιδ': 78, 'πρωθυπουργ': 73, 'κοινοβουλευτ': 73}
{'νε': 13265, 'δημοκρατ': 12138, 'τροπολογ': 7789, 'εθν': 7509, 'αρχ': 7014, 'αναπτυξ': 6460, 'ερωτης': 6266, 'σχεδ': 6120, 'κοινοβουλευτ': 5582, 'ευρ': 5278, 'συνεδριας': 5205, 'σωμ': 5108, 'αντιπολιτευς': 4893, 'υπηρες': 4716, 'στοιχει': 4629, 'ρυθμις': 4609, 'εκπροσωπ': 4550, 'τομ': 4541, 'δικαιωμ': 4489}
{'δημοκρατ': 298, 'νε': 276, 'εθν': 170, 'ανθρωπ': 158, 'τροπολογ': 149, 'ευρ': 145, 'εκατομμυρ': 143, 'ερωτης': 135, 'τραπεζ': 133, 'αναπτυξ': 125, 'αντιπολιτευς': 121, 'αμυν': 112, 'ανεξαρτητ': 111, 'δισεκατομμυ

### Keywords per parliament member

In [3]:
#get distinct parliament member names in a list
member_names = database.distinct("member_name")

In [4]:
keywords_by_member = kw.keywords_per_member(member_names, database)

Member αβδελας κωνσταντινου αποστολος
Member αβραμακης αντωνιου ελευθεριος
Member αβραμιδης νικολαου γαβριηλ
Member αβραμοπουλος λαμπρου δημητριος
Member αγαθοπουλου δημητριου ειρηνη-ελενη
Member αγατσα αριστειδη αριαδνη
Member αγγελης στεφανου ανεστης
Member αγγελοπουλος κωνσταντινου νικολαος
Member αγγελουσης αθανασιου ευσταθιος
Member αγγουρακης γεωργιου χαραλαμπος
Member αγοραστος αναστασιου κωνσταντινος
Member αδαμοπουλος αθανασιου γεωργιος
Member αδαμοπουλου αθανασιου αγγελικη
Member αδαμου στεργιου κωνσταντινα
Member αδρακτας φωτιου παναγιωτης
Member αηδονης ευαγγελου χρηστος
Member αθανασιαδης θεοδωρου αλεξανδρος
Member αθανασιος ηλιοπουλος
Member αθανασιος μικρουτσικος
Member αθανασιος σκορδας
Member αθανασιος τσαυταρης
Member αθανασιου ευαγγελου μαρια
Member αθανασιου ευθυμιου αθανασιος (νασος)
Member αθανασιου χριστοφα χαραλαμπος
Member αθανασοπουλος ανδρεα νικολαος
Member αιβαλιωτης ιωαννη κωνσταντινος
Member αιβατιδης γεωργιου ιωαννης
Member αικατερινη νοτοπουλου
Member ακ

In [5]:
print(keywords_by_member['αβδελας κωνσταντινου αποστολος'])

{'κυκλ': 111, 'συνεδριας': 88, 'σωμ': 71, 'ερωτης': 69, 'δυτικ': 57, 'κοινοβουλευτ': 55, 'επικαιρ ': 50, 'επικαιρ ερωτης ': 48}


In [7]:
import pickle

#with open('member_keywords.pickle', 'wb') as handle:
#    pickle.dump(keywords_by_member, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('member_keywords.pickle', 'rb') as handle:
#    mm = pickle.load(handle)


### Keywords per member per government

In [2]:
#get distinct member names
members = database.distinct("member_name")
governments_per_member = kw.keywords_per_member_per_government(members, database)

Member nan
Member αβδελας κωνσταντινου αποστολος
Member αβραμακης αντωνιου ελευθεριος
Member αβραμιδης νικολαου γαβριηλ
Member αβραμοπουλος λαμπρου δημητριος
Member αγαθοπουλου δημητριου ειρηνη-ελενη
Member αγατσα αριστειδη αριαδνη
Member αγγελης στεφανου ανεστης
Member αγγελοπουλος κωνσταντινου νικολαος
Member αγγελουσης αθανασιου ευσταθιος
Member αγγουρακης γεωργιου χαραλαμπος
Member αγοραστος αναστασιου κωνσταντινος
Member αδαμοπουλος αθανασιου γεωργιος
Member αδαμοπουλου αθανασιου αγγελικη
Member αδαμου στεργιου κωνσταντινα
Member αδρακτας φωτιου παναγιωτης
Member αηδονης ευαγγελου χρηστος
Member αθανασιαδης θεοδωρου αλεξανδρος
Member αθανασιος ηλιοπουλος
Member αθανασιος μικρουτσικος
Member αθανασιος σκορδας
Member αθανασιος τσαυταρης
Member αθανασιου ευαγγελου μαρια
Member αθανασιου ευθυμιου αθανασιος (νασος)
Member αθανασιου χριστοφα χαραλαμπος
Member αθανασοπουλος ανδρεα νικολαος
Member αιβαλιωτης ιωαννη κωνσταντινος
Member αιβατιδης γεωργιου ιωαννης
Member αικατερινη νοτοπουλο

In [3]:
#import pickle

#with open('keywords_per_member_per_government.pickle', 'wb') as handle:
#    pickle.dump(governments_per_member, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
governments_per_member['αθανασιος τσαυταρης']

{"['σαμαρα κ. αντωνιου(21/06/2012-26/01/2015)']": {'γεωργ': 52,
  'αγροτ': 47,
  'ευρ': 37,
  'παραγωγ': 36,
  'προϊοντα': 36,
  'τοπ': 34,
  'κτηνοτροφ': 32}}

In [2]:
import pickle
with open('keywords_per_member_per_government.pickle', 'rb') as handle:
    keywords_per_member_per_government = pickle.load(handle)

In [11]:
govs = list(database.distinct("government"))

In [12]:
print(govs[1])

['ζολωτα ξενοφωντα(23/11/1989-11/04/1990)']


In [12]:
total = list(keywords_per_member_per_government.values())
print(list(total[123].values()))

[{'τροπολογ': 2, 'ρυθμις': 2, 'συμβουλ': 2}, {'αναπτυξ': 1, 'προγραμμ': 1}]


In [10]:
import plotly.graph_objects as go

animals=list(total[123].keys())

fig = go.Figure(data=[
    go.Bar(name=list(total[123].keys())[0], x=animals, y=[20, 14, 23]),
    go.Bar(name=list(total[123].keys())[1], x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

### Keywords per party per government

In [2]:
# GET DISTINCT POLITICAL PARTY NAMES
parties = database.distinct("political_party")
governments_per_party = kw.keywords_per_party_per_government(parties, database)

Party nan
Party ανεξαρτητοι (εκτος κομματος)
Party ανεξαρτητοι δημοκρατικοι βουλευτες
Party ανεξαρτητοι ελληνες - πανος καμμενος
Party ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια
Party αντιπολιτευση
Party βουλη
Party δημοκρατικη ανανεωση
Party δημοκρατικη αριστερα
Party δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστικο κινημα - δημοκρατικη αριστερα)
Party δημοκρατικο κοινωνικο κινημα
Party ελληνικη λυση - κυριακος βελοπουλος
Party ενωση κεντρωων
Party εξωκοινοβουλευτικός
Party κινημα αλλαγης
Party κομμουνιστικο κομμα ελλαδας
Party λαικη ενοτητα
Party λαικος ορθοδοξος συναγερμος
Party λαικος συνδεσμος - χρυση αυγη
Party μετωπο ευρωπαικης ρεαλιστικης ανυπακοης (μερα25)
Party νεα δημοκρατια
Party οικολογοι εναλλακτικοι (ομοσπονδια οικολογικων εναλλακτικων οργανωσεων)
Party πανελληνιο σοσιαλιστικο κινημα
Party πολιτικη ανοιξη
Party συνασπισμος ριζοσπαστικης αριστερας
Party συνασπισμος της αριστερας των κινηματων και της οικολογιας
Party το ποταμι


In [115]:
#import pickle

#with open('keywords_per_party_per_government.pickle', 'wb') as handle:
#    pickle.dump(governments_per_party, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [112]:
print(governments_per_party['το ποταμι'])

{"['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)']": {'τροπολογ': 292, 'δημοκρατ': 231, 'νε': 229, 'συνεδριας': 223, 'σωμ': 219, 'συριζ': 190, 'κυκλ': 178, 'αριθμ': 173, 'ερωτης': 169, 'ευρ': 144, 'αρχ': 142, 'κοινοβουλευτ': 141, 'επικαιρ ': 134, 'δυτικ': 127, 'θεωρει': 126}, "['τσιπρα π. αλεξιου(26/01/2015-27/08/2015)']": {'ερωτης': 56, 'ποταμ': 45, 'νε': 41, 'κυκλ': 35, 'τροπολογ': 32, 'αριθμ': 32, 'ευρ': 31, 'συνεδριας': 31, 'κοινοβουλευτ': 30, 'μην': 29, 'πρωθυπουργ': 29, 'ανθρωπ': 28, 'συριζ': 28}}


In [97]:
import pandas as pd
def create_words_per_gov_df(governments_per_party_dict:dict):
    all_words = []
    for keywords in list(governments_per_party_dict.values()):
        words = set(keywords.keys())
        all_words.extend(words)
    governments = []
    counts = []
    for gov in governments_per_party_dict.keys():
        government = [gov for _ in range(len(governments_per_party_dict[gov]))]
        count = list(governments_per_party_dict[gov].values())
        counts.extend(count)
        governments.extend(government)
        
    df = pd.DataFrame(list(zip(all_words, governments)),columns=["keywords","governments"])
    df['count'] = counts
            
    df = df.fillna(0)
    df.head()
    return df

df1 = create_words_per_gov_df(governments_per_party['δημοκρατικη ανανεωση'])
df1.head()

,keywords,governments,count
0,εκδοτ,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],10
1,αποψ,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],8
2,προεδρει,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],8
3,κανονισμ,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],8
4,συμβουλ,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],7


In [93]:
import plotly.express as px

fig = px.bar(df1, x="keywords", y="count", color="governments", title="Wide-Form Input")
fig.show()

In [118]:
df2 = create_words_per_gov_df(governments_per_party['πανελληνιο σοσιαλιστικο κινημα'])
df2.head()

,keywords,governments,count
0,στοιχει,['καραμανλη α. κωνσταντινου(10/03/2004-19/09/2...,2477
1,αυξης,['καραμανλη α. κωνσταντινου(10/03/2004-19/09/2...,1940
2,νε,['καραμανλη α. κωνσταντινου(10/03/2004-19/09/2...,1040
3,εθν,['καραμανλη α. κωνσταντινου(10/03/2004-19/09/2...,802
4,προγραμμ,['καραμανλη α. κωνσταντινου(10/03/2004-19/09/2...,775


In [119]:
fig = px.bar(df2, x="keywords", y="count", color="governments", title="Wide-Form Input")
fig.show()

In [113]:
df2 = create_words_per_gov_df(governments_per_party['το ποταμι'])
df2.head()

,keywords,governments,count
0,αριθμ,['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)'],292
1,συριζ,['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)'],231
2,επικαιρ,['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)'],229
3,αρχ,['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)'],223
4,ευρ,['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)'],219


In [114]:
fig = px.bar(df2, x="keywords", y="count", color="governments", title="Wide-Form Input")
fig.show()